# DenoiSeg Example: DSB 2018
This is an example notebook which illustrates how DenoiSeg should be trained. In this notebook we use a refined version of the Kaggle 2018 Data Science Bowl (DSB 2018) dataset. We already split the data into train and test images. From the train images we then extracted 3800 training and 670 validation patches of size 128x128. The test set contains 50 images. 


In [ ]:
# Here we are just importing some libraries which are needed to run this notebook.
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage

from denoiseg.models import DenoiSeg, DenoiSegConfig
from denoiseg.utils.misc_utils import combine_train_test_data, shuffle_train_data, augment_data
from denoiseg.utils.seg_utils import *
from denoiseg.utils.compute_precision_threshold import measure_precision

from csbdeep.utils import plot_history

import urllib
import os
import zipfile

## Downloading and  Data Loading
We created three versions of this dataset by adding Gaussian noise with zero mean and standard deviations 10 and 20. The dataset are marked with the suffixes n0, n10 and n20 accordingly.

In the next cell you can choose which `noise_level` you would like to investigate.

In [ ]:
# Choose the noise level you would like to look at:
# Values: 'n0', 'n10', 'n20'
noise_level = 'n20'

In [ ]:
# create a folder for our data
if not os.path.isdir('./data'):
    os.mkdir('data')
    
if noise_level == 'n0':
    link = 'https://owncloud.mpi-cbg.de/index.php/s/1WXxLSqbK0ZIxF5/download'
elif noise_level == 'n10':
    link = 'https://owncloud.mpi-cbg.de/index.php/s/dRc1AHcaH8mqeh7/download'
elif noise_level == 'n20':
    link = 'https://owncloud.mpi-cbg.de/index.php/s/hy6xSq82kCoqqSH/download'
else:
    print('This noise level does not exist for this dataset.')

# check if data has been downloaded already
zipPath="data/DSB2018_{}.zip".format(noise_level)
if not os.path.exists(zipPath):
    #download and unzip data
    data = urllib.request.urlretrieve(link, zipPath)
    with zipfile.ZipFile(zipPath, 'r') as zip_ref:
        zip_ref.extractall("data")

In [ ]:
# Loading of the training images
trainval_data =  np.load('data/DSB2018_{}/train/train_data.npz'.format(noise_level))
train_images = trainval_data['X_train'].astype(np.float32)
train_masks = trainval_data['Y_train']
val_images = trainval_data['X_val'].astype(np.float32)
val_masks = trainval_data['Y_val']

In [ ]:
print("Shape of train_images: {}".format(train_images.shape))
print("Shape of train_masks:  {}".format(train_masks.shape))
print("Shape of val_images:   {}".format(val_images.shape))
print("Shape of val_masks:    {}".format(val_masks.shape))

## Small Amounts of Annotated Training Data
With DenoiSeg we present a solution to train deep neural networks if only few annotated ground truth segmentations are available. We simulate such a scenary by zeroing out all but a fraction of the available training data. In the next cell you can specify the percentage of training images for which ground truth annotations are available.

In [ ]:
# Set the number of annotated training images.
# Values: 0.0 (no annotated images) to total number of training images (all images have annotations)
number_of_annotated_training_images = 19
assert number_of_annotated_training_images >= 0.0 and number_of_annotated_training_images <=train_images.shape[0]

In [ ]:
# Seed to shuffle training data (annotated GT and raw image pairs).
seed = 1 

# First we shuffle the training images to remove any bias.
X_shuffled, Y_shuffled = shuffle_train_data(train_images, train_masks, random_seed=seed)

# Here we convert the number of annotated images to be used for training as percentage of available training data.
percentage_of_annotated_training_images = float((number_of_annotated_training_images/train_images.shape[0])*100.0)
assert percentage_of_annotated_training_images >= 0.0 and percentage_of_annotated_training_images <=100.0

# Here we zero out the segmentations of those training images which are not part of the selected annotated images.
X_frac, Y_frac = zero_out_train_data(X_shuffled, Y_shuffled, fraction = percentage_of_annotated_training_images)

# Now we apply data augmentation to the training patches:
# Rotate four times by 90 degree and add flipped versions.
X, Y_train_masks = augment_data(X_frac, Y_frac)
X_val, Y_val_masks = val_images, val_masks

# Here we add the channel dimension to our input images.
# Dimensionality for training has to be 'SYXC' (Sample, Y-Dimension, X-Dimension, Channel)
X = X[...,np.newaxis]
Y = convert_to_oneHot(Y_train_masks)
X_val = X_val[...,np.newaxis]
Y_val = convert_to_oneHot(Y_val_masks)
print("Shape of X:     {}".format(X.shape))
print("Shape of Y:     {}".format(Y.shape))
print("Shape of X_val: {}".format(X_val.shape))
print("Shape of Y_val: {}".format(Y_val.shape))

Next we look at a single sample. In the first column we show the input image, in the second column the background segmentation, in the third column the foreground segmentation and in the last column the border segmentation.

With the parameter `sample` you can choose different training patches. You will notice that not all of them have a segmentation ground truth.

In [ ]:
sample = 0
plt.figure(figsize=(20,5))
plt.subplot(1,4,1)
plt.imshow(X[sample,...,0])
plt.axis('off')
plt.title('Raw validation image')
plt.subplot(1,4,2)
plt.imshow(Y[sample,...,0], vmin=0, vmax=1, interpolation='nearest')
plt.axis('off')
plt.title('1-hot encoded background')
plt.subplot(1,4,3)
plt.imshow(Y[sample,...,1], vmin=0, vmax=1, interpolation='nearest')
plt.axis('off')
plt.title('1-hot encoded foreground')
plt.subplot(1,4,4)
plt.imshow(Y[sample,...,2], vmin=0, vmax=1, interpolation='nearest')
plt.axis('off')
plt.title('1-hot encoded border');

### Configure network parameters

In [ ]:
train_batch_size = 128
train_steps_per_epoch = max(100, min(int(X.shape[0]/train_batch_size), 400))

In the next cell, you can choose how much relative importance (weight) to assign to denoising 
and segmentation tasks by choosing appropriate value for `denoiseg_alpha` (between `0` and `1`; with `0` being
only segmentation and `1` being only denoising. Here we choose `denoiseg_alpha = 0.5`)

In [ ]:
conf = DenoiSegConfig(X, unet_kern_size=3, n_channel_out=4, relative_weights = [1.0,1.0,5.0],
                      train_steps_per_epoch=train_steps_per_epoch, train_epochs=200, 
                      batch_norm=True, train_batch_size=128, unet_n_first = 32, 
                      unet_n_depth=4, denoiseg_alpha=0.5, train_tensorboard=True)

vars(conf)

In [ ]:
model_name = 'DenoiSeg_DSB18_n20'
basedir = 'models'
model = DenoiSeg(conf, model_name, basedir)

In [ ]:
history = model.train(X, Y, (X_val, Y_val))

In [ ]:
history.history.keys()

In [ ]:
plot_history(history, ['loss', 'val_loss'])

## Computing Threshold Value
The network predicts 4 output channels:
1. The denoised input.
2. The background likelihoods.
3. The foreground likelihoods.
4. The border likelihoods.

We will threshold the foreground prediction image to obtain object segmentations. The optimal threshold is determined on the validation data. Additionally we can optimize the threshold for a given measure. In this case we choose the Average Precision (AP) measure.

In [ ]:
threshold, val_score = model.optimize_thresholds(val_images.astype(np.float32), val_masks, measure=measure_precision())

print("The higest score of {} is achieved with threshold = {}.".format(np.round(val_score, 3), threshold))

## Test Data
Finally we load the test data and run the prediction.

In [ ]:
test_data =  np.load('data/DSB2018_{}/test/test_data.npz'.format(noise_level), allow_pickle=True)
test_images = test_data['X_test']
test_masks = test_data['Y_test']

In [ ]:
predicted_images, precision_result = model.predict_label_masks(test_images, test_masks, 0.5, 
                                                                   measure=measure_precision())
print("Average precision over all test images with threshold = {} is {}.".format(0.5, np.round(precision_result, 3)))

### Visualize the results

In [ ]:
sl = -10
fig = plt.figure()
plt.figure(figsize=(20,10))
plt.subplot(1, 3, 1)
plt.imshow(test_images[sl])
plt.title("Raw image")

plt.subplot(1, 3, 2)
plt.imshow(predicted_images[sl])
plt.title("Predicted segmentation")

plt.subplot(1, 3, 3)
plt.imshow(test_masks[sl])
plt.title("Ground truth segmentation")

plt.show()

In [ ]:
print("Number of annotated images used for training:", number_of_annotated_training_images)
print("Noise level:", noise_level)
print("Considered alpha:", conf.denoiseg_alpha)

### Expected results for this dataset

<b>AP scores for DSB n0 dataset</b>.

||10 imgs|19 imgs|38 imgs|76 imgs|152 imgs|304 imgs|608 imgs|1216 imgs|2432 imgs|3800 imgs|
|--- |--- |--- |--- |--- |--- |--- |--- |--- |--- |--- |
|Alpha 0.5|0.625±0.008|0.637±0.01|0.663±0.01|0.678±0.004|0.694±0.006|0.742±0.007|0.769±0.006|0.794±0.003|0.775±0.004|0.785±0.002|
|Alpha 0.3|0.621±0.01|0.654±0.004|0.670±0.005|0.682±0.004|0.705±0.009|0.756±0.002|0.767±0.003|0.787±0.005|0.786±0.005|0.788±0.001|
|Alpha 0.7|0.612±0.01|0.643±0.01|0.666±0.005|0.674±0.005|0.699±0.004|0.725±0.004|0.774±0.005|0.786±0.006|0.791±0.001|0.784±0.005|
|Alpha best|0.652±0.005|0.667±0.002|0.684±0.002|0.697±0.004|0.746±0.007|0.770±0.001|0.787±0.003|0.802±0.001|0.799±0.002|0.801±0.001|

<b>AP scores for DSB n10 dataset</b>.

||10 imgs|19 imgs|38 imgs|76 imgs|152 imgs|304 imgs|608 imgs|1216 imgs|2432 imgs|3800 imgs|
|--- |--- |--- |--- |--- |--- |--- |--- |--- |--- |--- |
|Alpha 0.5|0.662±0.01|0.698±0.01|0.690±0.01|0.711±0.008|0.746±0.004|0.760±0.005|0.775±0.007|0.780±0.003|0.794±0.003|0.779±0.007|
|Alpha 0.3|0.675±0.009|0.682±0.006|0.710±0.004|0.720±0.005|0.744±0.006|0.760±0.007|0.779±0.003|0.792±0.002|0.796±0.002|0.795±0.002|
|Alpha 0.7|0.627±0.02|0.692±0.004|0.702±0.01|0.708±0.008|0.736±0.002|0.753±0.003|0.767±0.008|0.783±0.002|0.789±0.001|0.780±0.002|
|Alpha best|0.690±0.01|0.705±0.005|0.718±0.004|0.728±0.005|0.757±0.003|0.773±0.001|0.789±0.001|0.799±0.002|0.806±0.002|0.800±0.001|

<b>AP scores for DSB n20 dataset</b>.

||10 imgs|19 imgs|38 imgs|76 imgs|152 imgs|304 imgs|608 imgs|1216 imgs|2432 imgs|3800 imgs|
|--- |--- |--- |--- |--- |--- |--- |--- |--- |--- |--- |
|Alpha 0.5|0.677±0.01|0.689±0.008|0.697±0.008|0.714±0.002|0.723±0.006|0.733±0.004|0.741±0.003|0.766±0.006|0.775±0.003|0.771±0.006|
|Alpha 0.3|0.669±0.008|0.669±0.009|0.687±0.01|0.715±0.005|0.722±0.002|0.753±0.005|0.757±0.003|0.774±0.002|0.775±0.001|0.770±0.003|
|Alpha 0.7|0.627±0.03|0.667±0.01|0.702±0.006|0.698±0.008|0.718±0.006|0.733±0.005|0.747±0.004|0.767±0.003|0.770±0.004|0.773±0.003|
|Alpha best|0.690±0.006|0.702±0.003|0.710±0.006|0.723±0.003|0.738±0.001|0.757±0.004|0.766±0.003|0.782±0.001|0.789±0.002|0.788±0.001|

### Export your model for Fiji

In [ ]:
model.export_TF(name='DenoiSeg - DSB2018 Example', 
                description='This is the 2D DenoiSeg example trained on DSB2018 data in python.', 
                authors=["Tim-Oliver Buchholz", "Mangal Prakash", "Alexander Krull", "Florian Jug"],
                test_img=X_val[0,...,0], axes='YX',
                patch_shape=(128, 128))